# Deploy a Model or Bundle to an Endpoint

Welcome to this tutorial on deploying a model or a model bundle to a SambaNova dedicated endpoint!

Before you get started, please follow the set up instructions given in the [README](./README.md)

## 1.  Imports

In [1]:
import sys
sys.version

'3.11.11 (main, Dec 11 2024, 10:28:39) [Clang 14.0.6 ]'

In [2]:
from IPython.display import display, HTML
display(HTML("<style>:root { --jp-notebook-max-width: 100% !important; }</style>"))
import json
import os
from dotenv import load_dotenv
import pprint
load_dotenv()

True

In [3]:
import snsdk
snsdk.__file__

'/Users/varunk/anaconda3/envs/py_3_11_autogen/lib/python3.11/site-packages/snsdk/__init__.py'

In [4]:
from snsdk import SnSdk

## 2. Set up environment connector

Connects to the remote dedicated environment using the variables defined in `.env`

In [5]:
sn_env = SnSdk(host_url=os.getenv("SAMBASTUDIO_HOST_NAME"), 
                   access_key=os.getenv("SAMBASTUDIO_ACCESS_KEY"), 
                   tenant_id=os.getenv("SAMBASTUDIO_TENANT_NAME"))

## 3. Create or select a project

Projects are a way to organize endpoints and training/inference jobs

#### List available projects
You can list existing projects in which the endpoint can be created for model deployment

In [6]:
projects = sn_env.list_projects()["projects"]
sorted([project["name"] for project in projects])

['benchmarking', 'test_project_4']

#### Create a new project
If you do not wish to use an existing project, you may create a new one.

In [7]:
new_project = sn_env.create_project(
                    project_name="test_project",
                    description="A test project with a test endpoint"
                )

## 4. Select model or bundle to deploy

#### List models

Get the complete list of models. This includes models that are  
  - actually available
  - still in the process of uploading
  - exist in a remote storage from which they can be made available
  - not in a usable state

In [8]:
models = sn_env.list_models()["models"]
len(models)

140

Filter down to the models that are actually available on the environment

In [9]:
available_models = [m for m in models if m['status'] == 'Available']
len(available_models)

29

Print names of the available models

In [10]:
sorted([m["model_checkpoint_name"] for m in available_models])

['DeepSeek-R1',
 'DeepSeek-R1-Distill-Llama-70B',
 'DeepSeek-V3',
 'Meta-Llama-3-70B-Instruct',
 'Meta-Llama-3-8B-Instruct',
 'Meta-Llama-3.1-405B-Instruct',
 'Meta-Llama-3.1-70B-Instruct',
 'Meta-Llama-3.1-70B-SD-Llama-3.2-1B',
 'Meta-Llama-3.1-8B-Instruct',
 'Meta-Llama-3.2-1B-Instruct',
 'Meta-Llama-3.3-70B-Instruct',
 'Meta-Llama-3.3-70B-SD-Llama-3.2-1B-TP16',
 'Mistral-7B-Instruct-V0.2',
 'QwQ-32B-Preview',
 'QwQ-32B-Preview-SD-Qwen-2.5-QWQ-0.5B',
 'Qwen 2.5 72B TP16',
 'Qwen-2.5-72B-SD-Qwen-2.5-0.5B',
 'Qwen2-72B-Instruct',
 'Qwen2-7B-Instruct',
 'Qwen2.5-0.5B-Instruct',
 'Qwen2.5-0.5B-SFT-Instruct',
 'Qwen2.5-72B-Instruct',
 'Qwen2.5-7B-Instruct',
 'Salesforce--Llama-xLAM-2-70b-fc-r',
 'Salesforce--Llama-xLAM-2-8b-fc-r',
 'Samba-1 Turbo',
 'e5-mistral-7B-instruct',
 'meta-llama-3.1-70b',
 'qwen_llama_salesforce']

#### Select models to include in the bundle

In [11]:
selected_model = "qwen_llama_salesforce"

## Create endpoint

In [ ]:
endpoint_name = selected_model.lower().replace('_','-')
endpoint = sn_env.create_endpoint(
    project=new_project["name"],
    endpoint_name=endpoint_name,
    description="Endpoint for " + selected_model,
    model_checkpoint=selected_model,
    model_version=1,
    instances=1,
    hyperparams='{"model_parallel_rdus": "16", "num_tokens_at_a_time": "10"}',
    rdu_arch="SN40L-16",
    inference_api_openai_compatible=True
)

#### Check the status of the endpoint

In [17]:
endpoint = sn_env.endpoint_info(new_project["name"], endpoint_name)
endpoint['status']

'SettingUp'

## Get Endpoint Details
To test the endpoint, we will need to obtain some of its information. Note that this information can be obtained even while the model is setting up.

#### Get the endpoint URL

In [21]:
endpoint_url = endpoint['headers']['access-control-allow-origin'] + endpoint["url"]

#### Get the default endpoint API key
Note that:
  - New keys can be added using the `sn_env.add_endpoint_api_key` API.    
  - All keys can be revoked using the `sn_env.edit_endpoint_api_key` API.

In [24]:
endpoint_key = endpoint["api_keys"][0]["api_key"]

#### Get model names in the endpoint

In [25]:
endpoint_model_id = endpoint['targets'][0]["model"]
model_info = sn_env.model_info(endpoint_model_id, job_type="deploy")

#### Check if the model is standalone or composite (bundle)

In [28]:
model_info["type"]

'Composite'

#### If the model is a composite/bundle, list its constituents

In [30]:
model_constituents = [m["name"] for m in model_info["dependencies"]]
sorted(model_constituents)

['Meta-Llama-3.3-70B-Instruct',
 'Qwen-2.5-72B-SD-Qwen-2.5-0.5B',
 'Salesforce--Llama-xLAM-2-70b-fc-r',
 'Salesforce--Llama-xLAM-2-8b-fc-r']

## Test Endpoint
Once the endpoint is available, you can test it using the OpenAI API

#### Make sure endpoint is available

In [ ]:
endpoint = sn_env.endpoint_info(new_project["name"], endpoint_name)
endpoint['status']